In [1]:
from general_tools.notebook.gpu_utils import setup_one_gpu
GPU = 3
setup_one_gpu(GPU)

Picking GPU 3


In [2]:
import numpy as np
import os.path as osp

from geo_tool import Point_Cloud

from tf_lab.nips.helper import pclouds_centered_and_half_sphere, center_pclouds_in_unit_sphere
from tf_lab.iclr.helper import load_multiple_version_of_pcs
from tf_lab.data_sets.shape_net import snc_category_to_synth_id

from tf_lab.evaluate.generative_pc_nets import entropy_of_occupancy_grid, jensen_shannon_divergence, minimum_mathing_distance

In [3]:
%load_ext autoreload
%matplotlib inline
%autoreload 2

In [4]:
# Parameters
class_name = 'chair'
n_pc_samples = 2048
do_mmd = False
do_jsd = True

# For JSD
cmp_in_sphere = True
voxel_resolution = 28
use_uniform_prior = False

# For MMD
reduce_gt = 1000
batch_size = 1000

def identity(x):
    return x

if do_mmd:    
    pc_normalizer = identity
#     pclouds_with_zero_mean_in_unit_sphere
elif do_jsd:
    pc_normalizer = identity
    pc_normalizer = pclouds_centered_and_half_sphere
    pc_normalizer = center_pclouds_in_unit_sphere

In [5]:
# Load Wu Data, compute grid variable.
top_in_data = '/orions4-zfs/projects/optas/DATA/OUT/iclr/synthetic_samples/mit_wu_isosurface_extracted/'
wu_data = np.load(osp.join(top_in_data, class_name + '.npz'))
wu_data = pc_normalizer(wu_data[wu_data.keys()[0]])
assert(wu_data.shape[1] == n_pc_samples)

if do_jsd:
    _, wu_grid_var = entropy_of_occupancy_grid(wu_data, voxel_resolution, in_sphere=cmp_in_sphere)
    if use_uniform_prior:
        wu_grid_var += 1

In [6]:
# Load Ground-Truth Train Data (All data af the class.)
top_gt_dir = '/orions4-zfs/projects/optas/DATA/OUT/iclr/evaluations/gt_data/'

all_class_data = []
for split in ['train', 'test', 'val']:
    gt_file = osp.join(top_gt_dir, class_name + '_' + split + '.npz')
    gt_data = np.load(gt_file)  
    gt_data = pc_normalizer(gt_data[gt_data.keys()[0]])
    all_class_data.append(gt_data)

all_class_data = np.vstack(all_class_data)

In [7]:
jsd_samples = []
n_samples = 5
for _ in range(n_samples):
    n_wu = len(wu_data)
    gt_cmp_data = all_class_data[np.random.choice(np.arange(len(all_class_data)), n_wu)]
    if do_jsd:
        _, gt_grid_var = entropy_of_occupancy_grid(gt_cmp_data, voxel_resolution, in_sphere=cmp_in_sphere)
        if use_uniform_prior:
            gt_grid_var += 1
        res = jensen_shannon_divergence(wu_grid_var, gt_grid_var)
    jsd_samples.append(res)
print np.mean(jsd_samples)

0.170516077381


In [15]:
if do_mmd:
    if reduce_gt is not None:
        gt_cmp_data = gt_cmp_data[:reduce_gt]            
    res = minimum_mathing_distance(wu_data[:len(gt_cmp_data)], gt_cmp_data, batch_size, normalize=True, 
                                   verbose=True, use_sqrt=True)[0]
    print res

0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
0.204231
